In [2]:
import pandas as pd
import numpy as np

In [3]:
#corrected the Final Critic Item and made a new csv purchase_data1.csv
file = "Resources/purchase_data1.csv"
df_pymoli = pd.read_csv(file)

In [32]:
df_pymoli.head()
#df_pymoli.dtypes

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,133,Phaena87,20,Male,0,Splinter,1.28
1,150,Inasti31,17,Male,0,Splinter,1.28
2,204,Chamimla85,16,Male,0,Splinter,1.28
3,662,Mindilsa60,21,Male,0,Splinter,1.28
4,121,Ilosia37,23,Male,1,Crucifer,3.26


In [49]:
#count the number of players in column "SN" and set it to a variable
pymoli_total_players = df_pymoli["SN"].value_counts()
#count the number of rows in the value counts
pymoli_total_unique = pymoli_total_players.count()
#create a DataFrame for "Total Players"
df_total_players = pd.DataFrame({"Total Players": [pymoli_total_unique]})
df_total_players

,Total Players
0,576


In [34]:
df_Items = df_pymoli.drop_duplicates("Item ID")
unique_items = df_Items["Item ID"].count()
mean_price = round(df_pymoli["Price"].mean(),2)
total_revenue = df_pymoli["Price"].sum()
df_itemSummary = pd.DataFrame({"Number of Unique Items":[unique_items],
                              "Average Price":[mean_price],
                               "Number of Purchases":[df_pymoli_total_players],
                               "Total Revenue":[total_revenue]
                              })
#change the formating of the values to add $
df_itemSummary.style.format({"Average Price":"${:.2f}", 
                             "Total Revenue":"${:.2f}"})

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,182,$3.06,780,$2383.22


In [42]:
#Count the total number of Male, Female and Nonbinary Genders
df_pymoli_male = df_pymoli.loc[df_pymoli["Gender"] == "Male"]
gender_male_count = df_pymoli_male["Gender"].count()
df_pymoli_female = df_pymoli.loc[df_pymoli["Gender"] == "Female"]
gender_female_count = df_pymoli_female["Gender"].count()
df_pymoli_nonbinary = df_pymoli.loc[df_pymoli["Gender"] == "Other / Non-Disclosed"]
gender_nonbinary_count = df_pymoli_nonbinary["Gender"].count()

#calculate the percentages of each gender and round the result
percentage_male = round((gender_male_count / (pymoli_total_unique))*100,2)
percentage_female = round((gender_female_count / (pymoli_total_unique))*100,2)
percentage_nonbinary = round((gender_nonbinary_count / (pymoli_total_unique))*100,2)

#create a dataframe of each
df_itemSummary = pd.DataFrame({"Percentage of Players":[percentage_male, percentage_female, percentage_nonbinary],
                              "Total Count":[gender_male_count, gender_female_count, gender_nonbinary_count],
                              "Gender":["Male", "Female", "Other / Non-Disclosed"]
                              })
#set the index to the Gender columm
df_itemSummary = df_itemSummary.set_index("Gender")
#remove the index header
df_itemSummary.index.name = ""
df_itemSummary

,Percentage of Players,Total Count
,,
Male,113.19,652
Female,19.62,113
Other / Non-Disclosed,2.60,15


In [36]:
#calculate the Price summation and mean of each gender
df_pymoli_maleSum = df_pymoli_male["Price"].sum()
df_pymoli_femaleSum = df_pymoli_female["Price"].sum()
df_pymoli_nonbinarySum = df_pymoli_nonbinary["Price"].sum()
df_pymoli_maleMean = round(df_pymoli_male["Price"].mean(),2)
df_pymoli_femaleMean = round(df_pymoli_female["Price"].mean(),2)
df_pymoli_nonbinaryMean = round(df_pymoli_nonbinary["Price"].mean(),2)

#calculate the total spending of all genders
pymoli_purchaseTotals = df_pymoli_maleSum + df_pymoli_femaleSum + df_pymoli_nonbinarySum

#calculate the normalized values of each gender by dividing the summation and the count
pymoli_maleNormalized = round(df_pymoli_maleSum / gender_male_count,2)
pymoli_femaleNormalized = round(df_pymoli_femaleSum / gender_female_count,2)
pymoli_nonbinaryNormalized = round(df_pymoli_nonbinarySum / gender_nonbinary_count, 2)

#create a dataframe of the Purchase Count, Average Purchase price, Total Purchase Value, Normal Totals and Gender
df_priceSummary = pd.DataFrame({"Purchase Count":[gender_female_count, 
                                                  gender_male_count,  
                                                  gender_nonbinary_count],
                                
                              "Average Purchase Price":[df_pymoli_femaleMean, 
                                                        df_pymoli_maleMean, 
                                                        df_pymoli_nonbinaryMean],
                                
                              "Total Purchase Value":[df_pymoli_femaleSum, 
                                                      df_pymoli_maleSum, 
                                                      df_pymoli_nonbinarySum],
                                
                              "Normalized Totals":[pymoli_femaleNormalized, 
                                                   pymoli_maleNormalized, 
                                                   pymoli_nonbinaryNormalized],
                                
                              "Gender":["Female", "Male", "Other / Non-Disclosed"]
                              })
#set Gender as the index
df_priceSummary = df_priceSummary.set_index("Gender")
#remove the index headers
df_priceSummary.index.name = ""
#change the formating of the columns related to price
df_priceSummary.style.format({"Average Purchase Price":"${:.2f}", 
                              "Total Purchase Value":"${:.2f}", 
                              "Normalized Totals":"${:.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
,,,,
Female,113,$3.20,$361.94,$3.20
Male,652,$3.02,$1971.09,$3.02
Other / Non-Disclosed,15,$3.35,$50.19,$3.35


In [56]:
# Establish bins for ages and names for the bins
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
#create a new column to hold the binned values
df_pymoli["Percentage of Players"] = pd.cut(df_pymoli["Age"], age_bins, labels=group_names)

#group by the new bins and count how many are in each bin
df_pymoli.groupby("Percentage of Players")
df_pymoli_count = df_pymoli["Percentage of Players"].value_counts()
df_pymoli_percentage = round((df_pymoli_count / pymoli_total_unique)*100, 2)

#create a dataframe for the counts and the percentages
df1 = pd.DataFrame(df_pymoli_count)
df2 = pd.DataFrame(df_pymoli_percentage)

#merge the dataframes by their indexes 
df_ageDemos = df1.merge(df2, how='inner', left_index=True, right_index=True)

#reset the index and sort based on the index
df_ageDemos.reset_index(inplace=True)
df_ageDemos.sort_values(["index"], ascending=True)
df_ageDemos.set_index("index", inplace=True)
df_ageDemos.sort_index(inplace=True)

#rename the columns
df_ageDemos = df_ageDemos.rename(columns={"Percentage of Players_x": "Total Count",
                                          "Percentage of Players_y": "Percentage of Players",
                                         })
#reorder the columns
df_ageDemos = df_ageDemos[["Percentage of Players", "Total Count"]]
#remove the index header
df_ageDemos.index.name = ""
df_ageDemos

,Percentage of Players,Total Count
,,
<10,3.99,23
10-14,4.86,28
15-19,23.61,136
20-24,63.37,365
25-29,17.53,101
30-34,12.67,73
35-39,7.12,41
40+,2.26,13


In [57]:
#create a new column to bin the Age column values into age_bins and set the labels to group_names
df_pymoli["Percentage of Players"] = pd.cut(df_pymoli["Age"], age_bins, labels=group_names)
#set an age count df that groups the percentage of players and counts the values that fall into each bin
df_age_count = df_pymoli.groupby("Percentage of Players", as_index=False)["Age"].count()
#set a price mean df that groups the percentage of players and sets the Price column to the mean price of each age
df_price_mean = round(df_pymoli.groupby('Percentage of Players', as_index=False)['Price'].mean(),2)
df_price_sum = df_pymoli.groupby('Percentage of Players', as_index=False)['Price'].sum()

#merge the dataframes twice 
merged_df = pd.merge(df_price_sum, df_age_count, on="Percentage of Players", how="inner")
merged_df_2 = pd.merge(merged_df, df_price_mean, on="Percentage of Players", how="inner")

#calculate the normalized totals of Total Purchase Value and the Purchase Count
merged_df_2["Normalized Totals"] = round(merged_df_2["Price_x"]/merged_df_2["Age"],2)

#rename the columns
merged_df_3 = merged_df_2.rename(columns={"Price_x": "Total Purchase Value",
                                          "Price_y": "Average Purchase Price",
                                          "Age": "Purchase Count",
                                         })
#reorder the columns
merged_df_4 = merged_df_3[["Percentage of Players",
                           "Purchase Count",
                           "Average Purchase Price",
                           "Total Purchase Value",
                           "Normalized Totals"]]

#set the percentage of players to the index
merged_df_5 = merged_df_4.set_index("Percentage of Players")
#remove the headers
merged_df_5.index.name = ""
#adjust the format for dollars
merged_df_5.style.format({"Average Purchase Price":"${:.2f}", 
                              "Total Purchase Value":"${:.2f}", 
                              "Normalized Totals":"${:.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
,,,,
<10,23,$3.38,$77.82,$3.38
10-14,28,$3.01,$84.16,$3.01
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.06,$1115.44,$3.06
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


In [59]:
#calculate the sum of the Prices based on usernames and sort them by descending
df_pymoli_SN = df_pymoli.groupby("SN")["Price"].sum().reset_index()
df_pymoli_SN = df_pymoli_SN.sort_values("Price", ascending=False)

#count the number of times each username appears in the data
df_snCount = df_pymoli["SN"].value_counts()
#create a DataFrame for the usernames and the number of times they appear in the data
df_snCount_5 = pd.DataFrame({"Purchase Count":df_snCount})

#set the index to usernames
df_pymoli_SN = df_pymoli_SN.set_index("SN")

#merge the two dataframes based on index and then sort them based on price to be descending
merged_SN = pd.merge(df_snCount_5, df_pymoli_SN, left_index=True, right_index=True)
merged_SN = merged_SN.sort_values("Price", ascending=False)

#set the dataframe to the top 5 rows
merged_SN = merged_SN.iloc[0:5,0:2]
merged_SN["Average Purchase Price"] = round(merged_SN["Price"]/merged_SN["Purchase Count"],2)

#rename the columns
merged_SN_1 = merged_SN.rename(columns={"Price":"Total Purchase Value"})
#reorder the columns
merged_SN_2 = merged_SN_1[["Purchase Count", 
                           "Average Purchase Price", 
                           "Total Purchase Value"]]
#format for dollar signs
merged_SN_2 = merged_SN_2.style.format({"Average Purchase Price":"${:.2f}",
                              "Total Purchase Value":"${:.2f}"})
#remove the headers
merged_SN_2.index.name = "SN"
merged_SN_2

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [40]:
#group the Item IDs and Item Names and then find the summation and count
pitemsID = df_pymoli.groupby("Item ID").sum().reset_index()
pitemsName = df_pymoli.groupby("Item Name").sum().reset_index()
pitemsCount = df_pymoli.groupby("Item Name").count().reset_index()

#merge the dataframes based on Price and merge again based on Item name
pmerge1 = pd.merge(pitemsID, pitemsName, on="Price")
pmerge2 = pd.merge(pitemsCount, pmerge1, on="Item Name")

#sort base on Purchase IDs and then set to descending order
pmerge3 = pmerge2.sort_values("Purchase ID", ascending=False)

#reorder the columns
pmerge4 = pmerge3[["Item Name", "Item ID_x", "Purchase ID", "Price_y"]]

#rename the columns
pmerge5 = pmerge4.rename(columns={"Item ID_x": "Item ID",
                                          "Purchase ID": "Purchase Count",
                                          "Price_y": "Total Purchase Value",
                                         })

#calculate the Item Price by dividing the total purchases for each time by the total times the item was purchased
pmerge5["Item Price"] = pmerge5["Total Purchase Value"]/pmerge5["Purchase Count"]

#set the index to item id and item name
popularCount = pmerge5.set_index(["Item ID", "Item Name"])
#set the dataframe to the top 5 items
popularCount = popularCount.iloc[0:5,0:3]
#reorder the columns
popularCount = popularCount[["Purchase Count", 
                             "Item Price", 
                             "Total Purchase Value"]]
#format for dollar signs
popularCount = popularCount.style.format({"Item Price":"${:.2f}",
                              "Total Purchase Value":"${:.2f}"})
popularCount

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$63.44
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
82,Nirvana,9,$4.90,$44.10
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [41]:
pitemsID = df_pymoli.groupby("Item ID").sum().reset_index()
pitemsName = df_pymoli.groupby("Item Name").sum().reset_index()
pitemsCount = df_pymoli.groupby("Item Name").count().reset_index()

pmerge1 = pd.merge(pitemsID, pitemsName, on="Price")
pmerge2 = pd.merge(pitemsCount, pmerge1, on="Item Name")
pmerge3 = pmerge2.sort_values("Price_y", ascending=False)
pmerge4 = pmerge3[["Item Name", "Item ID_x", "Purchase ID", "Price_y"]]
pmerge5 = pmerge4.rename(columns={"Item ID_x": "Item ID",
                                          "Purchase ID": "Purchase Count",
                                          "Price_y": "Total Purchase Value",
                                         })
pmerge5["Item Price"] = pmerge5["Total Purchase Value"]/pmerge5["Purchase Count"]

cleanedItem = pmerge5.set_index(["Item ID", "Item Name"])
cleanedItem = cleanedItem.iloc[0:5,0:3]
cleanedItem_1 = cleanedItem[["Purchase Count", "Item Price", "Total Purchase Value"]]
cleanedItem_1 = cleanedItem_1.style.format({"Item Price":"${:.2f}",
                              "Total Purchase Value":"${:.2f}"})
cleanedItem_1

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$63.44
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
